### Libraries

In [41]:
import os
import sys
import itertools
import yaml
from patoolib import extract_archive

# Local Module Imports
sys.path.append("../src")  # adds source code directory
from utils import wkt_to_masc

### Main directories

In [17]:
current_dir = os.getcwd()
base_dir = os.path.dirname(current_dir)
dataset_dir = os.path.join(base_dir, "dataset")
data_dir = os.path.join(base_dir, "data")
config_file = os.path.join(base_dir, "config.yml")

#### Dataset extration and config file loading

In [18]:
# if there isn´t unrar installed, install it with: sudo apt-get install unrar (linux)
# for windows, install it from: https://www.rarlab.com/rar_add.htm (unrarw32.exe)

dataset_path = os.path.join(dataset_dir, "BurnedAreaUAV_dataset")
if not os.path.exists(dataset_path):
    extract_archive(
        os.path.join(dataset_dir, "BurnedAreaUAV_dataset_v1.rar"),
        program="unrar",
        outdir=dataset_dir,
    )
    os.remove(os.path.join(dataset_dir, "BurnedAreaUAV_dataset_v1.rar"))

with open(config_file, "r", encoding="utf-8") as f:
    config = yaml.safe_load(f)

#### Output directory struture creation

In [23]:
model_names = ["CVAE", "pstg", "shape"] 
keys = ["interpol", "extrapol"]
sub_keys = ["full", "sampled"]
sub_sub_keys = ["PNG", "WKT"]

for model_name in model_names:
    for key in (keys if model_name == "CVAE" else ["interpol"]):
        for sub_key, sub_sub_key in itertools.product(sub_keys, sub_sub_keys):
            # Create the directories
            dir_path = os.path.join(base_dir, "outputs", model_name, key, sub_key, sub_sub_key)
            os.makedirs(dir_path, exist_ok=True)
            # Create a .gitkeep file in the directory
            with open(os.path.join(dir_path, ".gitkeep"), "w") as f:
                pass

#### Masks directory creation

In [27]:
train_masks_dir = os.path.join(base_dir, config["data"]["train_dir"], "masks")
if not os.path.exists(train_masks_dir):
    os.makedirs(train_masks_dir)

test_masks_dir = os.path.join(base_dir, config["data"]["test_dir"], "masks")
if not os.path.exists(test_masks_dir):
    os.makedirs(test_masks_dir)

train_sampled_masks_dir = os.path.join(
    base_dir, config["data"]["train_sampled_dir"], "masks"
)
if not os.path.exists(train_sampled_masks_dir):
    os.makedirs(train_sampled_masks_dir)

### WKT files to masks connversion

In [40]:
# full training set
wkt_to_masc(
    wkt_file=os.path.join(base_dir, config["data"]["train_wkt"]),
    images_path=train_masks_dir,
    orig_dims=config["data"]["original_vid_dims"][::-1],
    height=config["data"]["input_size"][0],
    width=config["data"]["input_size"][1],
)

# test set
wkt_to_masc(
    wkt_file=os.path.join(base_dir, config["data"]["test_wkt"]),
    images_path=test_masks_dir,
    orig_dims=config["data"]["original_vid_dims"][::-1],
    height=config["data"]["input_size"][0],
    width=config["data"]["input_size"][1],
)

# sampled training set
wkt_to_masc(
    wkt_file=os.path.join(base_dir, config["data"]["sampled_masks_wkt"]),
    images_path=train_sampled_masks_dir,
    orig_dims=config["data"]["original_vid_dims"][::-1],
    height=config["data"]["input_size"][0],
    width=config["data"]["input_size"][1],
    )


INFO - 
    --------------------------------------
    # Properties of the resulting masks
    # Width: 512, Height: 512
    # Number of masks to create: 226
    --------------------------------------
    
100%|██████████| 226/226 [00:00<00:00, 588.33it/s]
INFO - 
    --------------------------------------
    # Properties of the resulting masks
    # Width: 512, Height: 512
    # Number of masks to create: 23
    --------------------------------------
    
100%|██████████| 23/23 [00:00<00:00, 632.64it/s]
INFO - 
    --------------------------------------
    # Properties of the resulting masks
    # Width: 512, Height: 512
    # Number of masks to create: 13
    --------------------------------------
    
100%|██████████| 13/13 [00:00<00:00, 534.88it/s]
